In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/dataset.zip >> /dev/null

In [3]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image
import numpy as np

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained EfficientNet-B0 model and remove the classification head
efficientnet = models.efficientnet_b0(pretrained=False)  # Set pretrained=False since we're loading custom weights
efficientnet.classifier = torch.nn.Sequential(*list(efficientnet.classifier.children())[:-1])  # Remove last layer
efficientnet.eval()  # Set model to evaluation mode
efficientnet.to(device)  # Move the model to GPU

# Load the saved model state (f10 from Task 1)
efficientnet.load_state_dict(torch.load('/content/f10_model_final.pth'))
efficientnet.eval()  # Ensure the model is in evaluation mode

# Load the saved prototypes for f10
class EuclideanLwPClassifier:
    def __init__(self, num_classes, feature_dim):
        self.num_classes = num_classes
        self.prototypes = np.zeros((num_classes, feature_dim))

    def calculate_prototypes(self, features, labels):
        for label in range(self.num_classes):
            class_features = features[labels == label]
            if len(class_features) > 0:
                self.prototypes[label] = class_features.mean(axis=0)

    def predict(self, features):
        distances = np.zeros((features.shape[0], self.num_classes))

        # Calculate Euclidean distance between each feature and the prototypes
        for label in range(self.num_classes):
            for i, feature in enumerate(features):
                distances[i, label] = np.linalg.norm(feature - self.prototypes[label])

        # Return the class with the minimum Euclidean distance
        return np.argmin(distances, axis=1)

    def update_prototypes(self, features, predicted_labels, alpha=0.7):
        for label in range(self.num_classes):
            class_features = features[predicted_labels == label]
            if len(class_features) > 0:
                # Update prototype with a mix of old and new information (alpha = 0.7)
                self.prototypes[label] = alpha * self.prototypes[label] + (1 - alpha) * class_features.mean(axis=0)

# Initialize LwP with number of classes (10) and feature dimension from EfficientNet-B0 (1280)
num_classes = 10
feature_dim = 1280  # EfficientNet-B0 feature dimension
lwp_model = EuclideanLwPClassifier(num_classes, feature_dim)

# Load prototypes from Task 1 (for f10)
lwp_model.prototypes = np.load('/content/f10_prototypes.npy')


# Define CIFAR-10 transform (same as Task 1)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to load dataset in mini-batches to manage memory usage
def load_train_dataset(filepath, transform, batch_size=64):
    """
    Load datasets from the given file path and apply transformations.
    """
    dataset = torch.load(filepath)
    print(f"Loaded dataset from {filepath}. Keys: {dataset.keys()}")

    if 'data' in dataset:
        data = dataset['data']
    else:
        raise KeyError("The dataset does not contain the required key 'data'.")

    if 'targets' in dataset:
        targets = dataset['targets']
    else:
        print("No 'targets' found in dataset. Using placeholder labels.")
        targets = np.zeros(len(data))  # Placeholder for unlabeled datasets

    data = [transform(Image.fromarray(img)) for img in data]
    dataset = TensorDataset(torch.stack(data), torch.tensor(targets, dtype=torch.long))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return dataloader

# Extract features in mini-batches
def extract_features(dataloader, model):
    features = []
    with torch.no_grad():  # Disable gradient calculation to save memory
        for data, _ in dataloader:
            data = data.to(device)  # Move data to GPU
            batch_features = model(data)  # Extract features from EfficientNet
            features.append(batch_features.cpu())  # Move features back to CPU if needed
    return torch.cat(features, dim=0)


# Define the evaluation function for Task 2 (held-out datasets)
def evaluate_task2_with_f11_to_f20(model, eval_paths_part_one, eval_paths_part_two, efficientnet_model, transform, num_datasets=10, start_from=11, num_task2_datasets=10):
    # Initialize the accuracy matrix for Task 2 (10 models x 20 datasets)
    accuracy_matrix_task2 = np.zeros((num_task2_datasets, num_datasets + num_task2_datasets))

    # Iterate over each new dataset from D'11 to D'20
    for i in range(start_from, start_from + num_task2_datasets):  # Starting from D'11 to D'20
        print(f"Training and Evaluating on Dataset D'{i}...")

        # Load and extract features for the current dataset (D'i) from eval_paths_part_two
        train_dataloader = load_train_dataset(eval_paths_part_two.format(i - start_from + 1), transform)  # Adjust indexing for part_two
        train_features = extract_features(train_dataloader, efficientnet_model).cpu().numpy()
        train_targets = [target.cpu().numpy() for _, target in train_dataloader]
        train_targets = np.concatenate(train_targets)

        # Update the LwP model with new dataset (D'i)
        lwp_model.update_prototypes(train_features, train_targets)

        # Predict labels and calculate accuracy for D'i with the updated model
        predicted_labels = lwp_model.predict(train_features)
        accuracy = (predicted_labels == train_targets).mean()

        # Store accuracy for D'i
        accuracy_matrix_task2[i - start_from, i - start_from] = accuracy  # Store accuracy for D'i (diagonal)

        # Evaluate the updated model on held-out datasets from D1 to D'i (previous datasets)
        for j in range(1, i + 1):  # Evaluate only on D1 to D_i for f_i (not all datasets)
            if j <= 10:  # Datasets from part_one (D1 to D10)
                eval_dataloader = load_train_dataset(eval_paths_part_one.format(j), transform)
            else:  # Datasets from part_two (D11 to D20)
                eval_dataloader = load_train_dataset(eval_paths_part_two.format(j - 10), transform)

            eval_features = extract_features(eval_dataloader, efficientnet_model).cpu().numpy()
            eval_targets = [target.cpu().numpy() for _, target in eval_dataloader]
            eval_targets = np.concatenate(eval_targets)

            # Predict labels and calculate accuracy
            predicted_labels = lwp_model.predict(eval_features)
            accuracy = (predicted_labels == eval_targets).mean()
            accuracy_matrix_task2[i - start_from, j - 1] = accuracy

            print(f"Accuracy on Dataset D^{j} after training with Model f{i}: {accuracy:.4f}")

    return accuracy_matrix_task2


# Task 2 Evaluation (held-out datasets)
accuracy_matrix_task2 = evaluate_task2_with_f11_to_f20(
    lwp_model,
    '/content/dataset/part_one_dataset/eval_data/{}_eval_data.tar.pth',  # Paths for D1 to D10 from part_one_dataset
    '/content/dataset/part_two_dataset/eval_data/{}_eval_data.tar.pth',  # Paths for D11 to D20 from part_two_dataset
    efficientnet,
    transform,
    num_datasets=10,  # D1 to D10
    start_from=11,     # Start from D'11 (after Task 1)
    num_task2_datasets=10  # Evaluate for D'11 to D'20
)

print("Accuracy Matrix for Task 2 (F11-F20):")
print(accuracy_matrix_task2)

# Optionally, save the accuracy matrix for Task 2
np.save('accuracy_matrix_task2.npy', accuracy_matrix_task2)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-3-09ae54bda9be>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

Training and Evaluating on Dataset D'11...
Loaded dataset from /content/dataset/part_two_dataset/eval_data/1_eval_data.tar.pth. Keys: dict_keys(['data', 'targets'])
Loaded dataset from /content/dataset/part_one_dataset/eval_data/1_eval_data.tar.pth. Keys: dict_keys(['data', 'targets'])
Accuracy on Dataset D^1 after training with Model f11: 0.8352
Loaded dataset from /content/dataset/part_one_dataset/eval_data/2_eval_data.tar.pth. Keys: dict_keys(['data', 'targets'])
Accuracy on Dataset D^2 after training with Model f11: 0.8432
Loaded dataset from /content/dataset/part_one_dataset/eval_data/3_eval_data.tar.pth. Keys: dict_keys(['data', 'targets'])
Accuracy on Dataset D^3 after training with Model f11: 0.8368
Loaded dataset from /content/dataset/part_one_dataset/eval_data/4_eval_data.tar.pth. Keys: dict_keys(['data', 'targets'])
Accuracy on Dataset D^4 after training with Model f11: 0.8464
Loaded dataset from /content/dataset/part_one_dataset/eval_data/5_eval_data.tar.pth. Keys: dict_key